# PARSING etherscan.io

I made an attempt to parse more full information about user's transactions.

In [58]:
%load_ext autoreload
%autoreload 2
%aimport

import requests
from bs4 import BeautifulSoup
from application.load_transaction_data import load_ether_data, load_token_data
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import random
import time
import warnings

warnings.filterwarnings("ignore")

transactions_df = load_ether_data()\
    .query("Status != 'Error(0)'")
token_df = load_token_data()

transactions_parsed_df = pd.read_pickle("transactions_parsed_df")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
all-except-skipped

Modules to skip:



In [69]:
expr = "TRANSFER  1,355.117744540135181807 Ether From Wrapped"
expr.split("TRANSFER")[1].split("Ether")[0].strip(" ")

'1,355.117744540135181807'

In [ ]:
from user_agent import generate_user_agent
def export_transfer_list(transfer: str):
    # try:
    #     amount = re.findall(r"(\d+\.\d+|\d+\,\d+)", transfer)[0]
    # except IndexError:
    #     amount = re.findall(r"\d+", transfer)[0]
    amount = float(transfer.split("TRANSFER")[1].split("Ether")[0].strip(" ").replace(",", ""))
    from_ = re.findall(r"From (.*?) To", transfer)[0]
    to_ = transfer.split("  ")[-1]
    return amount, from_, to_


def extract_token_info(token_url):
    result_dict = {}

    # Parse from and to
    try:
        result_dict['from'] =  token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-from tooltip-address').text
    except Exception as e:
        result_dict['from'] = e
    try:
        result_dict['to'] = token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-to tooltip-address').text
    except Exception as e:
        result_dict['to'] = e

    # Parse USD value. Check, where or not does USD value exist on this page?
    try:
        token1_usd = re.findall(r"\(\$.*?\)", str(token_url))[0]
        token1_usd = token1_usd.strip("()").replace(" (", "").replace("$", "")
        result_dict['usd'] = float(token1_usd.replace(",", "_"))
        result_dict['amount'] = token_url.find("span", 'data-toggle'=='tooltip').text
        result_dict['amount'] = float(result_dict['amount'].replace(",", ""))
    except (AttributeError, IndexError) as e:
        result_dict['usd'] = 'empty'
        result_dict['amount'] = token_url.find_all("span", class_='mr-1')[-1].text
        result_dict['amount'] = float(result_dict['amount'].replace(",", ""))

    # Parse token hash in order to parse full info later
    try:
        result_dict['token_hash'] = token_url.find("a").get("href").split("?")[0].split("/")[2]
    except Exception as e:
        result_dict['token_hash'] = e
    return result_dict

def extract_tokens_info(transaction_hash):
    req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers={"User-Agent": generate_user_agent()})
    content = req.content
    soup = BeautifulSoup(content)
    tokens_df = []
    bad_tokens = []
    errors = []
    all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
    for index, tokens in enumerate(all_tokens):
        try:
            tokens_df.append(
                pd.DataFrame.from_dict(extract_token_info(all_tokens[index]), orient='index').T
            )
        except Exception as e:
            bad_tokens.append(index)
            errors.append(e)
    try:

        tokens_df = pd.concat(tokens_df)
    except ValueError:
        tokens_df = pd.DataFrame(data=[transaction_hash], columns=['Txhash'])

    tokens_df.index = np.arange(tokens_df.shape[0])
    # TRANSFER parsing
    transfer_text_list = []
    transfers = soup.find_all("li", class_='media align-items-baseline')
    if len(transfers) > 0:
        for transf in transfers:
            transfer_text_list.append(transf.text.replace("\xa0", ""))

        transfer_df = pd.DataFrame(list(map(export_transfer_list, transfer_text_list)), columns=['transfer_amount', 'transfer_from', 'transfer_to'])
        tokens_df = pd.concat([tokens_df, transfer_df], axis=1)

    tokens_df['Txhash'] = transaction_hash
    return tokens_df

all_tokens_df = []
transactions_list = set(transactions_df['Txhash'])
for tx_hash in tqdm(transactions_list):
    try:
        time.sleep(random.uniform(0, 1))
        all_tokens_df.append(extract_tokens_info(tx_hash))
    except Exception as e:
        print(e)
        print(f"Error on {tx_hash}")
    pd.concat(all_tokens_df).to_pickle("transactions_transfers_df")
# extract_tokens_info("0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfda319d4a260f358196c4b")

 15%|█▍        | 1110/7418 [19:24<2:01:13,  1.15s/it]

could not convert string to float: '1 wei From Wrapped'
Error on 0x90c67a86473a56f543d0984e569e60f4b70ea85a623342c822bc0b4d5821ce7e


 15%|█▌        | 1129/7418 [19:46<1:41:46,  1.03it/s]

In [74]:
transactions_parsed_df.query("Txhash == '0x52b25346387d60c526de43ec5ce71574852cbe40bcb2cec8aed9b40ce7d52996'")


,Txhash,from,to,usd,amount,token_hash,transfer_amount,transfer_from,transfer_to,transfer_amount_prep,ETH_amount


In [77]:
transactions_parsed_df

,Txhash,from,to,usd,amount,token_hash,transfer_amount,transfer_from,transfer_to,transfer_amount_prep,ETH_amount
0,0x4c3b415bb98370a9ef572567a4e13b61759763c08192...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: Exchange Proxy,Balancer: ETH/USDC 50/50 #7,24.06,0.06177,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.1,Balancer: Exchange Proxy,Wrapped Ether,0.1,-0.1
1,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: ETH/USDC 50/50 #7,Balancer: Exchange Proxy,24.12,24.136325,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,NaN,NaN,NaN,NaN,NaN
2,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: Exchange Proxy,Balancer: PEAK/USDC/ETH 30/40/30 #2,8.9,0.022847,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,NaN,NaN,NaN,NaN
3,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,Balancer: PEAK/USDC/ETH 30/40/30 #2,Balancer: Exchange Proxy,8.94,8.94618,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
0,0xacdcf3b9346cd26b0eb80ee6d000bf463b9eee29ec7e...,NaN,NaN,NaN,NaN,NaN,500,Compound: cETH,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,500.0,500.0
0,0x624b45fa83aa2506e5accac1f6708c702159b826ef08...,Compound: Comptroller,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,10.53,0.079894,0xc00e94cb662c3520282e6f5717214004a7f26888,647,Compound: cETH,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,647.0,647.0
1,0x624b45fa83aa2506e5accac1f6708c702159b826ef08...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Compound: cETH Token,399162.61,32302.250466,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,NaN,NaN,NaN,NaN,NaN
0,0xc3499213c05aff45629def03167afa1845fa67803c71...,NaN,NaN,NaN,NaN,NaN,500,Compound: cETH,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,500.0,500.0


In [59]:
transactions_parsed_df_new = pd.read_pickle("transactions_parsed_df_new")
transactions_parsed_df = pd.concat([transactions_parsed_df, transactions_parsed_df_new])
transactions_parsed_df['transfer_amount_prep'] = transactions_parsed_df['transfer_amount'].apply(lambda x: x.replace(",", "") if type(x) == str else x)
transactions_parsed_df['transfer_amount_prep'] = pd.to_numeric(transactions_parsed_df['transfer_amount_prep'], errors='coerce')
transactions_parsed_df['ETH_amount'] = np.where(transactions_parsed_df['transfer_to'] == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962',
                                                transactions_parsed_df['transfer_amount_prep'],
                                                -transactions_parsed_df['transfer_amount_prep'])

In [63]:
delete = transactions_parsed_df.merge(transactions_df[['Txhash', 'DateTime', "Quantity"]], how='left', on='Txhash')
delete.query("DateTime >= '2020-09-02' & DateTime < '2020-09-03'")

,Txhash,from,to,usd,amount,token_hash,transfer_amount,transfer_from,transfer_to,transfer_amount_prep,ETH_amount,DateTime,Quantity
964,0x52b25346387d60c526de43ec5ce71574852cbe40bcb2...,NaN,NaN,NaN,NaN,NaN,5.117744540135181807,Wrapped Ether,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,5.117745,5.117745,2020-09-02 00:02:03,-0.000000
965,0x2d50f9a2f00594c941af9271ba400f604964234cdf40...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-02 00:23:40,-0.000000
966,0xc96f019a1bfb8d736a709bbfa63d5fe792c6b34434dc...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Uniswap V2: USDC,308855.07,309274.356884,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,8.496251403084912011,Uniswap V2: Router 2,Wrapped Ether,8.496251,-8.496251,2020-09-02 00:31:49,-648.496251
967,0xc96f019a1bfb8d736a709bbfa63d5fe792c6b34434dc...,Uniswap V2: Router 2,Uniswap V2: USDC,286295.37,648.496251,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaN,NaN,NaN,NaN,NaN,2020-09-02 00:31:49,-648.496251
968,0xc96f019a1bfb8d736a709bbfa63d5fe792c6b34434dc...,Null Address: 0x000…000,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,2386495.75,0.011705,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,NaN,NaN,NaN,NaN,NaN,2020-09-02 00:31:49,-648.496251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,0xc4def72e7ba1a323aa786c0240c65be59afbf269a25d...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Kimchi Finance: Staking Pool,2429755.75,0.011919,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,NaN,NaN,NaN,NaN,NaN,2020-09-02 18:07:41,-0.000000
19097,0x7d6eeb70890f0399b0fc771601c2e165753caaaf3e99...,Uniswap V2: Router 2,Uniswap V2: SUSHI,18541.98,42.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,42,Uniswap V2: Router 2,Wrapped Ether,42.000000,-42.000000,2020-09-02 16:41:19,-42.000000
19098,0x7d6eeb70890f0399b0fc771601c2e165753caaaf3e99...,Uniswap V2: SUSHI,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,20680.47,2864.777247,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,NaN,NaN,NaN,NaN,NaN,2020-09-02 16:41:19,-42.000000
19396,0xc1da604638595e6c1dcdc1ee221acd3d0f353557648a...,Uniswap V2: Router 2,Uniswap V2: USDC,286959.24,650.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,650,Uniswap V2: Router 2,Wrapped Ether,650.000000,-650.000000,2020-09-02 00:10:01,-650.000000


In [60]:
find_eth_deals_df = transactions_df[['Txhash', 'DateTime', "Quantity"]]\
    .merge(transactions_parsed_df\
            .query("transfer_to == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962' | transfer_from == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962'")
           .groupby(['Txhash'], as_index=False)['ETH_amount'].sum(),
           on=['Txhash'], how='left')\
    .merge(transactions_df[['Txhash', 'TxnFee(ETH)']], on='Txhash', how="left")
find_eth_deals_df['ETH_final'] = np.where(find_eth_deals_df['Quantity'] != 0, find_eth_deals_df['Quantity'], find_eth_deals_df['ETH_amount'])
find_eth_deals_df['ETH_net'] = find_eth_deals_df['ETH_final'] - find_eth_deals_df['TxnFee(ETH)']
find_eth_deals_df['ETH_cumsum'] = find_eth_deals_df['ETH_net'].cumsum()

In [62]:
find_eth_deals_df

,Txhash,DateTime,Quantity,ETH_amount,TxnFee(ETH),ETH_final,ETH_net,ETH_cumsum
0,0x4c3b415bb98370a9ef572567a4e13b61759763c08192...,2020-08-04 20:21:33,1.000000,NaN,0.000882,1.000000,0.999118,0.999118
1,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,2020-08-04 20:26:40,-0.100000,NaN,0.014775,-0.100000,-0.114775,0.884343
2,0x88877e0aff3d534341cbb11d9fa19e5c807a164efd3f...,2020-08-05 18:14:02,9.156451,NaN,0.001050,9.156451,9.155401,10.039743
3,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...,2020-08-05 18:18:49,-0.000000,NaN,0.004291,NaN,NaN,NaN
4,0x12c77d689b0a62785e4ab18999382e8ce24c2b927bad...,2020-08-05 19:01:33,2228.526819,NaN,0.000983,2228.526819,2228.525836,2238.565580
...,...,...,...,...,...,...,...,...
7428,0x2e2926cccdc525856f0aa64e70917ffbec4a02f557f1...,2022-04-24 00:36:21,-0.000000,NaN,0.011870,NaN,NaN,NaN
7429,0xbc81557cb35bb4eda725803c320eee011b06e92a05af...,2022-04-24 00:37:11,-0.000000,7.509397,0.013450,7.509397,7.495947,-719819.760839
7430,0x167c46dd943aa9ff360ff99fbf2683fc6e78c377f472...,2022-04-24 00:38:04,-0.000000,NaN,0.007490,NaN,NaN,NaN
7431,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,2022-04-24 00:40:23,-0.000000,NaN,0.012496,NaN,NaN,NaN


In [61]:
find_eth_deals_df.set_index("DateTime")['ETH_net'].resample("D").sum().cumsum()

DateTime
2020-08-04         0.884343
2020-08-05         8.563874
2020-08-06         8.563874
2020-08-07         8.563874
2020-08-08         9.031854
                  ...      
2022-04-20   -719715.167537
2022-04-21   -719715.167537
2022-04-22   -719715.167537
2022-04-23   -719715.167537
2022-04-24   -719773.867370
Freq: D, Name: ETH_net, Length: 629, dtype: float64

In [41]:
find_eth_deals_df

,Txhash,DateTime,Quantity,ETH_amount,TxnFee(ETH),ETH_final,ETH_net,ETH_cumsum
0,0x4c3b415bb98370a9ef572567a4e13b61759763c08192...,2020-08-04 20:21:33,1.000000,0.0,0.000882,1.000000,0.999118,0.999118
1,0xdf08f1b6048a3c151737d797c5a5da5892cff66dfdfd...,2020-08-04 20:26:40,-0.100000,-0.1,0.014775,-0.100000,-0.114775,0.884343
2,0x88877e0aff3d534341cbb11d9fa19e5c807a164efd3f...,2020-08-05 18:14:02,9.156451,0.0,0.001050,9.156451,9.155401,10.039743
3,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...,2020-08-05 18:18:49,-0.000000,0.0,0.004291,0.000000,-0.004291,10.035453
4,0x12c77d689b0a62785e4ab18999382e8ce24c2b927bad...,2020-08-05 19:01:33,2228.526819,0.0,0.000983,2228.526819,2228.525836,2238.561289
...,...,...,...,...,...,...,...,...
7536,0x2e2926cccdc525856f0aa64e70917ffbec4a02f557f1...,2022-04-24 00:36:21,-0.000000,0.0,0.011870,0.000000,-0.011870,-731383.057506
7537,0xbc81557cb35bb4eda725803c320eee011b06e92a05af...,2022-04-24 00:37:11,-0.000000,0.0,0.013450,0.000000,-0.013450,-731383.070956
7538,0x167c46dd943aa9ff360ff99fbf2683fc6e78c377f472...,2022-04-24 00:38:04,-0.000000,0.0,0.007490,0.000000,-0.007490,-731383.078446
7539,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,2022-04-24 00:40:23,-0.000000,0.0,0.012496,0.000000,-0.012496,-731383.090942


In [98]:
transactions_parsed_df = pd.read_pickle("transactions_parsed_df")
transactions_parsed_user_df = transactions_parsed_df[(transactions_parsed_df["from"] == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962') |
                                                        (transactions_parsed_df['to'] == '0x99fd1378ca799ed6772fe7bcdc9b30b389518962')]
transactions_parsed_user_df['amount_prep'] = transactions_parsed_user_df['amount'].apply(lambda x: x.replace(",", "") if type(x) == str else x)
transactions_parsed_user_df['amount_prep'] = pd.to_numeric(transactions_parsed_user_df['amount_prep'], errors='coerce')

In [109]:
transactions_parsed_user_df.query("Txhash == '0x1f0e87327d9ca8df226b4a098c4438fc406f3fc1516785cdc93339b81122d169'")

,Txhash,from,to,usd,amount,token_hash,transfer_amount,transfer_from,transfer_to,amount_prep
0,0x1f0e87327d9ca8df226b4a098c4438fc406f3fc15167...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Uniswap V2: KIMCHI 2,2032.19,1462.055355,0x1e18821e69b9faa8e6e75dffe54e7e25754beda0,7.350894404996841542,Wrapped Ether,Uniswap V2: Router 2,1462.055355


In [108]:
transactions_parsed_user_df.dropna(subset=['transfer_to'])

,Txhash,from,to,usd,amount,token_hash,transfer_amount,transfer_from,transfer_to,amount_prep
0,0x83ad137f056d6aceb989349e982f238ae0a84e62225c...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Balancer: Exchange Proxy,185.1,847.41,0x408e41876cccdc0f92210600ef50372656052a38,0.490367183219354705,Wrapped Ether,Balancer: Exchange Proxy,847.410000
0,0x6b6456129efc6da90ff1ad25a372bc42c6e38381a1b9...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Balancer: Exchange Proxy,2892.63,942.15,0xd533a949740bb3306d119cc777fa900ba034cd52,6.497547074746942237,Wrapped Ether,Balancer: Exchange Proxy,942.150000
0,0xa510b919b4425ac57bb0df6efa780d1aad68cd3461dc...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Balancer: Exchange Proxy,299.58,200.0,0x45f24baeef268bb6d63aee5129015d69702bcdfa,0.224910549619237661,Wrapped Ether,Balancer: Exchange Proxy,200.000000
0,0xc96f019a1bfb8d736a709bbfa63d5fe792c6b34434dc...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Uniswap V2: USDC,308855.07,309274.356884,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,8.496251403084912011,Uniswap V2: Router 2,Wrapped Ether,309274.356884
0,0x5b7ce6889a298f749f6a030c24d72f94607802a0db78...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Uniswap V2: KIMCHI 2,1617.98,1164.051969,0x1e18821e69b9faa8e6e75dffe54e7e25754beda0,5.869872362761801408,Uniswap V2: Router 2,Wrapped Ether,1164.051969
...,...,...,...,...,...,...,...,...,...,...
0,0x6f94a07c15a843c8c7da05b38ad55635e7f3a4dba1d1...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x: Exchange Proxy Flash Wallet,153346.58,6707.532651,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,1.382664866456094883,0xb576491f1e6e5e62f1d8f26062ee822b40b0e0d4,0x: Exchange Proxy Flash Wallet,6707.532651
0,0x9de41530e6454db794232ae5bc8efce6164a7da99599...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x: Exchange Proxy Flash Wallet,127922.15,57777.048212,0xd533a949740bb3306d119cc777fa900ba034cd52,2.506999006346713577,0x8301ae4fc9c624d1d396cbdaa1ed877821d7c511,0x: Exchange Proxy Flash Wallet,57777.048212
0,0x7e7d08c1421b5adb5bd322738858f2c1e1c3920a11f1...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x: Exchange Proxy Flash Wallet,177398.92,65939.22386,0xd533a949740bb3306d119cc777fa900ba034cd52,9.888795775759164741,0x8301ae4fc9c624d1d396cbdaa1ed877821d7c511,0x: Exchange Proxy Flash Wallet,65939.223860
0,0x9064b75f900ee7bb958360f6e8b249d3b59a8fbc8ae8...,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x: Exchange Proxy Flash Wallet,168979.27,7391.322046,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,4.739058047800032071,0xb576491f1e6e5e62f1d8f26062ee822b40b0e0d4,0x: Exchange Proxy Flash Wallet,7391.322046


## Parsing token hash

In [67]:
all_token_hash = [i for i in set(all_tokens_user_df['token_hash'].unique()) if type(i) == str and len(i) == 42]
def parsing_tokens(token_hash):
    token_dict = {}
    token_dict['token_hash'] = token_hash
    req = requests.get(f"https://etherscan.io/token/{token_hash}",
                   headers={"User-Agent": generate_user_agent()})
    content = req.content
    soup = BeautifulSoup(content)
    try:
        token_dict['token_short_name'] = soup.find("span", class_='text-secondary small').text
    except Exception as e:
        token_dict['token_short_name'] = e
    try:
        token_dict['token_long_name'] = soup.find("a", class_='mb-1 mb-sm-0 u-label u-label--xs u-label--info').text
    except Exception as e:
        token_dict['token_long_name'] = e
    return token_dict

token_names_df = []
for token in tqdm(all_token_hash):
    time.sleep(random.uniform(0, 2))
    try:
        token_names_df.append(
                pd.DataFrame.from_dict(parsing_tokens(token), orient='index').T)
    except Exception as e:
        token_names_df.append(pd.DataFrame(data=[token], columns=['Txhash']))
token_names_df = pd.concat(token_names_df)
token_names_df.to_pickle("token_hash_name")

# Add token name
all_tokens_df_2 = all_tokens_df_2.merge(token_names_df[['token_hash', 'token_short_name']], how='left', on='token_hash')

100%|██████████| 241/241 [05:56<00:00,  1.48s/it]
